In [0]:
!pip install -qqq python-dotenv
import dotenv
from dotenv import load_dotenv
_ = load_dotenv()

In [0]:
!pip install -qqq llama_index

In [0]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional

def get_doc_tools(file_path: str, name:str) -> str:
    ''' Get Vector query and Summary query from the document.'''
    #load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)
    summary_index = SummaryIndex(nodes)
    def vector_query(query:str,
                     page_numbers: Optional[List[str]])-> str:
        """Use to answer questions over the MetaGPT paper.
    
        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.
    
        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE 
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.
        
        """
        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key":"page_label", "value": p} for p in page_numbers
        ]
        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response
    vector_query_tool = FunctionTool.from_defaults(
        name=f"{name}_Vector_Query",
        fn=vector_query)
    
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True)
    summary_tool = QueryEngineTool.from_defaults(
        name=f"{name}_Summary_Query",
        query_engine = summary_query_engine,
        description = ("Use ONLY IF you want to get a holistic summary of MetaGPT."
                        " Do NOT use if specific questions over MetaGPT"),
    )
    return vector_query_tool, summary_tool

In [0]:
vector_tool, summary_tool = get_doc_tools("meta-gpt-paper.pdf", "MetaGPT")

In [0]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model='gpt-3.5-turbo', temperature=0)

In [0]:
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

In [0]:
import nest_asyncio
nest_asyncio.apply()

In [0]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [0]:
response = agent.query("Tell me about the agent rols in MetaGPT"
                       "and then how they communicate with each other")

In [0]:
print (response.source_nodes[1].get_content(metadata_mode='all'))

# Memory

In [0]:
response = agent.chat(
  "Tell me about the evaluation datasets used")

In [0]:
response = agent.chat("tell me the results over one of the datasets")

# Agent Control

In [0]:
agent_worker = FunctionCallingAgentWorker.from_tools(
  [vector_tool, summary_tool],
  llm = llm,
  verbose=True,
                                          )
agent = AgentRunner(agent_worker)

In [0]:
task = agent.create_task("Tell me about agent rols in MetaGPT"
                         "and then how they communicate with each other")

In [0]:
step_output = agent.run_step(task.task_id)

In [0]:
completed_steps = agent.get_completed_steps(task.task_id)
print (f'Num completed for task {task.task_id}:{len(completed_steps)}')
print (completed_steps[0].output.sources[0].raw_output)

In [0]:
upcoming_steps = agent.get_upcoming_steps(task.task_id) 
print (f'Num upcoming for task {task.task_id}:{len(upcoming_steps)}')
upcoming_steps[0]

In [0]:
next_step = agent.run_step(task.task_id,input="How about how agents share information?")


In [0]:
step_output  = agent.run_step(task.task_id)
print (step_output.is_last)

In [0]:
completed_steps = agent.get_completed_steps(task.task_id)
print (f'Num completed for task {task.task_id}:{len(completed_steps)}')
print (completed_steps[0].output.sources[0].raw_output)